# Lesson 8.5: Advanced Agent Design Patterns with LangGraph

---

In previous lessons, we built basic Agents with LangGraph and understood how they reason, act, and manage state. However, the real world often demands more sophisticated Agents capable of human interaction, handling complex situations, and even self-correction. This lesson will introduce **advanced Agent design patterns** with LangGraph, extending the capabilities of your LLM applications.

## 1. Human-in-the-Loop: Integrating Human Intervention

In many cases, an Agent should not operate completely autonomously. Human-in-the-Loop (HITL) intervention is necessary to:

* **Validation:** Ensure the Agent makes correct decisions or actions before executing critical tasks (e.g., sending emails, performing financial transactions).
* **Information Provision:** When the Agent lacks information or cannot retrieve it, it can ask the user to provide it.
* **Correction:** When the Agent makes a mistake or enters an undesirable loop, a human can intervene to correct it.

### Implementing Human-in-the-Loop with LangGraph

In LangGraph, you can implement HITL by:

1.  **Adding a "Human Review" Node:** This Node will pause the Agent's flow and await input from a human.
2.  **Conditional Edge:** After the "Human Review" Node, a Conditional Edge will decide the next flow based on human feedback (e.g., "approved" to continue, "rejected" to go back to a previous step, or "provide_info" to process new information).




---

## 2. Tool Calling: Handling Multiple Tool Calls and Result Aggregation

Real-world Agents often need to use multiple tools in one reasoning turn or need to aggregate information from multiple sources.

* **Calling Multiple Tools in One Turn:** An LLM might decide to call multiple tools simultaneously if those tasks are independent or can run in parallel.
    * **LangChain's `tool_calling_agent`:** Modern Agents in LangChain (e.g., `create_tool_calling_agent`) can automatically handle this if the LLM supports multi-tool calling (e.g., GPT-4, Gemini).
    * **In LangGraph:** Your LLM Node will return a list of `AgentAction`s (if the LLM supports it), and your `call_tool` Node will need to be modified to execute all those actions and collect the results.
* **Aggregating Results:** After calling multiple tools, the results (Observations) need to be aggregated and fed back to the LLM so it can generate a coherent final response. This is often done by adding all `ToolMessage`s to the `agent_scratchpad` or `chat_history` in the state.




---

## 3. Self-Correction/Self-Healing Agents: Designing Agents that Automatically Detect and Fix Errors

Self-correcting Agents are those capable of recognizing when they make mistakes or go off track, and automatically adjusting their behavior.

* **Error Detection Mechanisms:**
    * **Output Analysis:** The LLM can be instructed to analyze its own output or the output of tools to look for signs of errors (e.g., parsing errors, empty search results, irrelevant responses).
    * **Condition Checks:** Conditional Nodes can be added to verify business constraints or data validity.
* **Error Correction Mechanisms:**
    * **Retry with Different Prompt:** If the LLM generates invalid output, it can be asked to retry with a refined prompt, providing more specific formatting instructions.
    * **Retry with Different Tool:** If a tool fails, the Agent can try an alternative tool.
    * **Change Strategy:** If one strategy is ineffective, the Agent can revert to the reasoning step and choose a different approach.

### Implementing Self-Correction with LangGraph

LangGraph is ideal for self-correction due to its looping and Conditional Edges capabilities:

1.  **"Error Detection" Node:** A Node that checks the output of the previous Node.
2.  **Conditional Edge:** If an error is detected, a conditional edge will route the flow to an "Error Handling" Node or back to the LLM Node with the error message in the state so the LLM can re-reason.




---

## 4. Multi-Agent Systems (Introduction): Concept of Multiple Agents Interacting

When a problem is too large or complex for a single Agent, we can use a **Multi-Agent System**.

* **Concept:** Multiple Agents (each potentially specializing in a specific domain or task) interact, collaborate, and communicate with each other to solve a larger problem.
* **Roles:**
    * **Task Delegation:** A primary Agent can delegate smaller tasks to specialized Agents.
    * **Information Exchange:** Agents exchange information and intermediate results with each other.
    * **Coordination:** Agents coordinate actions to achieve an overall goal.
* **Applications:** Complex research, project planning, social simulations.


### Implementing Multi-Agent Systems with LangGraph

LangGraph can be used to build multi-Agent systems by:

1.  **Each Agent as a Subgraph:** Each Agent can be an independent LangGraph graph.
2.  **"Delegate" Node:** A Node in one Agent's graph can call another Agent's graph.
3.  **Shared State/Communication:** Agents can communicate by updating a common state or through message passing mechanisms.




---

## 5. Practical Example: Building a More Complex Agent with Self-Correction Capabilities or Human Confirmation

We will extend the Agent from Lesson 8.4 to add a simple self-correction capability: if the LLM attempts to call a non-existent tool, the Agent will recognize the error and attempt to correct its action.

**Preparation:**
* Ensure you have the necessary libraries installed: `langchain-openai`, `google-search-results`, `numexpr`, `langgraph`.
* Set the `OPENAI_API_KEY` and `SERPAPI_API_KEY` environment variables.

In [ ]:
# Install libraries if not already installed
# pip install langchain-openai openai google-search-results numexpr langgraph

import os
from typing import TypedDict, Annotated, List, Union, Dict, Any
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, ToolMessage
from langgraph.graph import StateGraph, END
import operator
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.utilities import SerpAPIWrapper
from langchain.tools import Tool
from langchain_community.tools.calculator.tool import Calculator
from langchain_core.agents import AgentFinish, AgentAction # To parse LLM output

# Thiết lập biến môi trường cho khóa API của OpenAI và SerpAPI
# os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"
# os.environ["SERPAPI_API_KEY"] = "YOUR_SERPAPI_API_KEY"

# --- 1. Định nghĩa kiểu trạng thái cho đồ thị Agent ---
class AgentState(TypedDict):
    chat_history: Annotated[List[BaseMessage], operator.add]
    intermediate_steps: Annotated[List[Union[AgentAction, ToolMessage]], operator.add]
    # Thêm trường để theo dõi số lần thử lại (cho self-correction)
    retry_count: int

# --- 2. Khởi tạo LLM và Tools ---
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

search_tool = Tool(
    name="Google Search",
    func=SerpAPIWrapper().run,
    description="Hữu ích khi bạn cần tìm kiếm thông tin trên Google về các sự kiện hiện tại hoặc dữ liệu thực tế." # Useful when you need to search for information on Google about current events or factual data.
)
calculator_tool = Calculator()
tools = [search_tool, calculator_tool]

# --- 3. Định nghĩa Prompt cho Agent ---
agent_prompt = ChatPromptTemplate.from_messages([
    ("system", "Bạn là một trợ lý hữu ích. Bạn có quyền truy cập vào các công cụ sau: {tools}. Sử dụng chúng để trả lời các câu hỏi của người dùng. Nếu bạn đã có câu trả lời cuối cùng, hãy trả lời trực tiếp. Nếu bạn gặp lỗi với một công cụ, hãy thử lại hoặc thử một cách tiếp cận khác."), # You are a helpful assistant. You have access to the following tools: {tools}. Use them to answer user questions. If you have a final answer, respond directly. If you encounter an error with a tool, try again or try a different approach.
    MessagesPlaceholder(variable_name="chat_history"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

# --- 4. Định nghĩa các Node của đồ thị ---

# Node 1: Call LLM (phần suy luận/hành động của ReAct)
def call_llm_node(state: AgentState) -> Dict[str, Any]:
    """
    Node này gọi LLM để suy luận về bước tiếp theo.
    LLM có thể quyết định gọi Tool hoặc đưa ra câu trả lời cuối cùng.
    """
    print("\n--- Node: Call LLM (Suy luận/Hành động) ---") # --- Node: Call LLM (Reasoning/Acting) ---
    messages = agent_prompt.format_messages(
        tools=tools,
        chat_history=state["chat_history"],
        agent_scratchpad=state["intermediate_steps"]
    )
    
    response = llm.invoke(messages)
    
    if "Final Answer:" in response.content:
        final_answer_content = response.content.split("Final Answer:", 1)[1].strip()
        return {"chat_history": [AIMessage(content=final_answer_content)], "intermediate_steps": [AgentFinish(return_values={"output": final_answer_content}, log=response.content)]}
    
    try:
        # Cải thiện phân tích AgentAction một chút
        # Đây vẫn là một điểm yếu nếu LLM không tuân thủ định dạng chính xác.
        # Trong thực tế, bạn có thể dùng LangChain's AgentOutputParser.
        thought_part = ""
        action_part = ""
        action_input_part = ""

        if "Thought:" in response.content:
            parts = response.content.split("Thought:", 1)
            thought_part = parts[1].split("Action:", 1)[0].strip() if "Action:" in parts[1] else parts[1].strip()
        
        if "Action:" in response.content:
            parts = response.content.split("Action:", 1)[1].split("Action Input:", 1)
            action_part = parts[0].strip()
            action_input_part = parts[1].strip() if len(parts) > 1 else ""

        action = AgentAction(tool=action_part, tool_input=action_input_part, log=response.content)
        print(f"  LLM quyết định hành động: Tool='{action.tool}', Input='{action.tool_input}'") # LLM decides to act:
        return {"intermediate_steps": [action]}

    except Exception as e:
        print(f"  Lỗi phân tích đầu ra LLM thành Action: {e}. Phản hồi LLM: {response.content}") # Error parsing LLM output to Action:
        # Nếu LLM trả về định dạng không thể phân tích, chúng ta sẽ coi đây là một lỗi
        # và cập nhật retry_count, sau đó quay lại LLM để nó tự sửa.
        return {
            "chat_history": [AIMessage(content=f"Lỗi phân tích phản hồi của tôi. Vui lòng thử lại.")], # Error parsing my response. Please try again.
            "intermediate_steps": [AgentFinish(return_values={"output": "LLM parsing error"}, log=response.content)],
            "retry_count": state.get("retry_count", 0) + 1 # Tăng số lần thử lại # Increment retry count
        }


# Node 2: Call Tool (phần hành động của ReAct)
def call_tool_node(state: AgentState) -> Dict[str, Any]:
    """
    Node này thực thi Tool mà LLM đã quyết định.
    Bao gồm logic xử lý lỗi Tool và cập nhật retry_count.
    """
    print("--- Node: Call Tool (Thực thi công cụ) ---") # --- Node: Call Tool (Execute tool) ---
    last_action = state["intermediate_steps"][-1]
    
    tool_name = last_action.tool
    tool_input = last_action.tool_input

    selected_tool = next((t for t in tools if t.name == tool_name), None)
    if selected_tool:
        try:
            tool_output = selected_tool.run(tool_input)
            print(f"  Tool '{tool_name}' trả về: {tool_output[:100]}...") # Tool '{tool_name}' returns:
            return {"intermediate_steps": [ToolMessage(content=tool_output, tool_call_id=last_action.tool)], "retry_count": 0} # Reset retry_count khi thành công # Reset retry_count on success
        except Exception as e:
            error_message = f"Lỗi khi thực thi Tool '{tool_name}' với đầu vào '{tool_input}': {e}" # Error executing Tool '{tool_name}' with input '{tool_input}':
            print(f"  {error_message}")
            # Nếu Tool thất bại, trả về lỗi và tăng retry_count
            return {"intermediate_steps": [ToolMessage(content=error_message, tool_call_id=last_action.tool)], "retry_count": state.get("retry_count", 0) + 1} # If Tool fails, return error and increment retry_count
    else:
        # Nếu Tool không tồn tại, đây là một lỗi mà LLM cần tự sửa
        error_message = f"Lỗi: Không tìm thấy Tool có tên '{tool_name}'. Vui lòng kiểm tra lại tên công cụ." # Error: Tool with name '{tool_name}' not found. Please check tool name again.
        print(f"  {error_message}")
        return {"intermediate_steps": [ToolMessage(content=error_message, tool_call_id="unknown_tool")], "retry_count": state.get("retry_count", 0) + 1} # If Tool does not exist, this is an error that LLM needs to self-correct

# --- 5. Định nghĩa hàm điều kiện cho Conditional Edge (quyết định tiếp tục vòng lặp hay kết thúc) ---
def should_continue(state: AgentState) -> str:
    """
    Hàm này kiểm tra bước trung gian cuối cùng và retry_count để quyết định luồng tiếp theo.
    """
    last_step = state["intermediate_steps"][-1]
    retry_count = state.get("retry_count", 0)
    
    MAX_RETRIES = 2 # Số lần thử lại tối đa # Max retries

    if isinstance(last_step, AgentFinish):
        print("--- Quyết định: KẾT THÚC (AgentFinish) ---") # --- Decision: END (AgentFinish) ---
        return "end"
    elif isinstance(last_step, AgentAction):
        # Nếu LLM đưa ra một Action, chúng ta sẽ gọi Tool
        print("--- Quyết định: TIẾP TỤC (AgentAction) ---") # --- Decision: CONTINUE (AgentAction) ---
        return "continue"
    elif isinstance(last_step, ToolMessage):
        # Nếu bước cuối cùng là một ToolMessage (tức là kết quả của Tool)
        # và có lỗi (retry_count > 0), quay lại LLM để tự sửa
        if "Lỗi" in last_step.content or retry_count > 0: # Kiểm tra nội dung lỗi hoặc retry_count # Check error content or retry_count
            print(f"--- Quyết định: TỰ SỬA LỖI (ToolMessage lỗi, thử lại lần {retry_count}) ---") # --- Decision: SELF-CORRECT (ToolMessage error, retry attempt {retry_count}) ---
            if retry_count >= MAX_RETRIES:
                print("--- Đã đạt số lần thử lại tối đa. KẾT THÚC ---") # --- Max retries reached. END ---
                return "end" # Kết thúc nếu quá nhiều lần thử lại # End if too many retries
            return "call_llm" # Quay lại LLM để nó suy nghĩ lại # Return to LLM to re-reason
        else:
            # Nếu ToolMessage thành công, quay lại LLM để tổng hợp hoặc kết thúc
            print("--- Quyết định: TIẾP TỤC (ToolMessage thành công) ---") # --- Decision: CONTINUE (ToolMessage successful) ---
            return "call_llm"
    else:
        print(f"--- Quyết định: LỖI/KHÔNG XÁC ĐỊNH (Kiểu không mong muốn: {type(last_step)}) ---") # --- Decision: ERROR/UNKNOWN (Unexpected type:
        return "end"

# --- 6. Xây dựng đồ thị Agent ---
workflow = StateGraph(AgentState)

# Thêm các Node
workflow.add_node("call_llm", call_llm_node)
workflow.add_node("call_tool", call_tool_node)

# Đặt điểm bắt đầu: Luôn bắt đầu bằng việc gọi LLM để suy luận
workflow.set_entry_point("call_llm")

# Định nghĩa cạnh có điều kiện từ Node LLM
workflow.add_conditional_edges(
    "call_llm",
    should_continue,
    {
        "continue": "call_tool",
        "end": END,
        "call_llm": "call_llm" # Thêm cạnh này để LLM có thể tự gọi lại chính nó nếu cần tự sửa lỗi phân tích # Add this edge so LLM can call itself again if it needs to self-correct parsing errors
    }
)

# Định nghĩa cạnh từ Node Tool quay trở lại Node LLM
workflow.add_edge("call_tool", "call_llm")

# Biên dịch đồ thị
app = workflow.compile()

print("\n--- Bắt đầu thực hành Agent với khả năng tự sửa lỗi ---") # --- Starting Agent with self-correction capabilities ---

# --- Tình huống 1: Câu hỏi cần tìm kiếm và tính toán (bình thường) ---
print("\n--- Tình huống 1: Câu hỏi cần tìm kiếm và tính toán (bình thường) ---") # --- Scenario 1: Question requires search and calculation (normal) ---
initial_state_1 = {"chat_history": [HumanMessage(content="Thời tiết hôm nay ở London là bao nhiêu độ C? Sau đó nhân kết quả với 2.")], "intermediate_steps": [], "retry_count": 0} # What is the weather like today in London in Celsius? Then multiply the result by 2.
final_state_1 = app.invoke(initial_state_1)
print(f"\nPhản hồi cuối cùng:") # Final response:
for message in final_state_1["chat_history"]:
    print(f"{message.type.capitalize()}: {message.content}")

# --- Tình huống 2: LLM cố gắng gọi một công cụ không tồn tại (tự sửa lỗi) ---
print("\n--- Tình huống 2: LLM cố gắng gọi một công cụ không tồn tại (tự sửa lỗi) ---") # --- Scenario 2: LLM attempts to call a non-existent tool (self-correction) ---
# Để mô phỏng, chúng ta sẽ tạm thời thay đổi prompt để LLM cố tình gọi một tool sai
original_prompt = agent_prompt
agent_prompt = ChatPromptTemplate.from_messages([
    ("system", "Bạn là một trợ lý hữu ích. Bạn có quyền truy cập vào các công cụ sau: {tools}. Sử dụng chúng để trả lời các câu hỏi của người dùng. Nếu bạn đã có câu trả lời cuối cùng, hãy trả lời trực tiếp. Cố gắng sử dụng công cụ 'NonExistentTool' nếu bạn không chắc chắn."), # Intentionally make LLM try a wrong tool
    MessagesPlaceholder(variable_name="chat_history"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])
# Cần biên dịch lại app sau khi thay đổi prompt
agent = create_react_agent(llm, tools, agent_prompt)
app = StateGraph(AgentState)
app.add_node("call_llm", call_llm_node)
app.add_node("call_tool", call_tool_node)
app.set_entry_point("call_llm")
app.add_conditional_edges("call_llm", should_continue, {"continue": "call_tool", "end": END, "call_llm": "call_llm"})
app.add_edge("call_tool", "call_llm")
app = app.compile()

initial_state_2 = {"chat_history": [HumanMessage(content="Hãy nói về LangChain.")], "intermediate_steps": [], "retry_count": 0} # Tell me about LangChain.
final_state_2 = app.invoke(initial_state_2)
print(f"\nPhản hồi cuối cùng:") # Final response:
for message in final_state_2["chat_history"]:
    print(f"{message.type.capitalize()}: {message.content}")

# Khôi phục prompt gốc
agent_prompt = original_prompt
agent = create_react_agent(llm, tools, agent_prompt)
app = StateGraph(AgentState)
app.add_node("call_llm", call_llm_node)
app.add_node("call_tool", call_tool_node)
app.set_entry_point("call_llm")
app.add_conditional_edges("call_llm", should_continue, {"continue": "call_tool", "end": END, "call_llm": "call_llm"})
app.add_edge("call_tool", "call_llm")
app = app.compile()

print("\n--- Kết thúc thực hành Agent với khả năng tự sửa lỗi ---") # --- End of Agent with self-correction capabilities practical ---
